In [1]:
import pandas as pd
import numpy as np
import time
import re
import warnings

# Suppress pandas warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')


# --- MODULE 1: DATA PREPARATION AND CLEANING ---

def prepare_data(file_path: str) -> pd.DataFrame:
    """
    Loads raw CSV data, performs robust cleaning, date parsing,
    and initial filtering for options and trading hours.
    
    Args:
        file_path (str): Path to the options data CSV.
        
    Returns:
        pd.DataFrame: A cleaned, time-indexed DataFrame ready for backtesting.
    """
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"FATAL ERROR: Options data file not found at {file_path}. Check file path.")
        raise

    # 1.1 Robust Data Cleaning and Renaming
    df.columns = df.columns.str.strip()
    df.rename(columns={'Call/Put': 'Option_Type_Raw'}, inplace=True)

    # 1.2 Time Series Indexing (Combined Date and Time)
    # Uses dayfirst=True for D-M-Y format robustness
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True, errors='coerce')
    df.dropna(subset=['Datetime'], inplace=True)
    df.set_index('Datetime', inplace=True)
    df = df.sort_index()
    df['Trade_Date'] = df.index.date
    
    # 1.3 Feature Engineering: Extracting Strike
    def extract_strike(ticker):
        match = re.search(r'(\d+)(CE|PE)', str(ticker))
        return int(match.group(1)) if match else np.nan

    df['Strike_Price'] = df['Ticker'].apply(extract_strike)
    df['Option_Type'] = df['Option_Type_Raw']

    # CRITICAL FIX: Ensure all price columns are numeric
    price_cols = ['Open', 'High', 'Low', 'Close']
    for col in price_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 1.4 Final Trading Dataframe (Filtering for Trading Hours/Relevant Tickers)
    df_trading = df.loc[
        # Only filter by CE/PE type and core trading hours
        (df['Option_Type'].isin(['CE', 'PE'])) & 
        (df.index.time >= pd.to_datetime('09:15:00').time()) & 
        (df.index.time <= pd.to_datetime('15:30:00').time()),
        ['Ticker', 'High', 'Close', 'Strike_Price', 'Option_Type', 'Trade_Date']
    ].copy()

    df_trading.dropna(subset=['Ticker'], inplace=True)
    
    if df_trading.empty:
        raise Exception("Data Preparation failed: No valid option data found after filtering.")

    return df_trading


# --- MODULE 2: STRATEGY EXECUTION AND P&L CALCULATION ---

def execute_strangle_strategy(df_trading: pd.DataFrame, params: dict) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Executes the short strangle strategy and calculates trade results and daily P&L.

    Args:
        df_trading (pd.DataFrame): Cleaned options data.
        params (dict): Dictionary of strategy parameters (ENTRY_TIME, STOP_LOSS_MULTIPLIER, etc.).

    Returns:
        tuple[pd.DataFrame, pd.DataFrame]: (df_trades_results, df_daily_pnl)
    """
    ENTRY_TIME = params['ENTRY_TIME']
    EXIT_TIME = params['EXIT_TIME']
    STOP_LOSS_MULTIPLIER = params['STOP_LOSS_MULTIPLIER']
    LOT_SIZE = params['LOT_SIZE']
    
    # 2.1 Identify Expiry Days (Wednesdays)
    trade_dates = df_trading['Trade_Date'].unique()
    wednesdays = pd.Series(trade_dates)
    wednesdays = wednesdays[pd.to_datetime(wednesdays).dt.dayofweek == 2].tolist() 

    # 2.2 Select Entry Options (5-Minute Window Search & Max Premium)
    trades_list = []
    entry_start_time = pd.to_datetime(ENTRY_TIME).time()
    entry_end_time = pd.to_datetime('09:25:00').time() # 5-minute search window

    for date in trade_dates:
        df_day_all = df_trading[df_trading['Trade_Date'] == date]
        
        # Filter data for the entire 5-minute entry window
        df_day_entry_window = df_day_all[
            (df_day_all.index.time >= entry_start_time) &
            (df_day_all.index.time <= entry_end_time)
        ]
        
        # Filter for valid tradable prices
        df_day = df_day_entry_window[(df_day_entry_window['Close'].notna()) & (df_day_entry_window['Close'] > 0)]
        
        daily_trades = []

        for opt_type in ['CE', 'PE']:
            selection = df_day[df_day['Option_Type'] == opt_type].copy()
            
            if not selection.empty:
                # Select the single row corresponding to the HIGHEST premium
                entry = selection.loc[selection['Close'].idxmax()]
                
                # Robustness check for DataFrame instead of Series (multiple maxes)
                if isinstance(entry, pd.DataFrame):
                    entry = entry.iloc[0]

                entry_timestamp = entry.name # Ticker index is the datetime
                
                daily_trades.append({
                    'Trade_Date': date,
                    'Ticker': entry.get('Ticker'),
                    'Entry_Price': entry.get('Close'), 
                    'Entry_Time_Actual': entry_timestamp.time(),
                    'Option_Type': opt_type
                })

        if len(daily_trades) == 2: # Must have both CE and PE
            trades_list.extend(daily_trades) 

    df_trades_long = pd.DataFrame(trades_list)

    if df_trades_long.empty:
        raise Exception("Backtest halted: Cannot calculate performance on zero trading days. Check data compatibility.")

    # 2.4 Calculate Stop Loss Threshold (SL Price)
    df_trades_long['SL_Price'] = df_trades_long['Entry_Price'] * STOP_LOSS_MULTIPLIER

    # --- 3. P&L Calculation ---

    # 3.1 Isolate Trading Window Data (for Max High Check)
    min_entry_time = df_trades_long['Entry_Time_Actual'].min()
    df_window = df_trading[
        (df_trading.index.time >= min_entry_time) &
        (df_trading.index.time <= pd.to_datetime(EXIT_TIME).time())
    ].copy()

    df_window['Ticker'] = df_window['Ticker'].astype(str)
    df_window['Trade_Date'] = pd.to_datetime(df_window['Trade_Date']) 

    # 3.2 Vectorized SL Check: Find the highest observed price (Max High)
    df_sl_check = df_window.groupby(['Trade_Date', 'Ticker']).agg(
        Max_High_in_Window=('High', 'max')
    ).reset_index()

    df_trades_long['Ticker'] = df_trades_long['Ticker'].astype(str)
    df_trades_long['Trade_Date'] = pd.to_datetime(df_trades_long['Trade_Date']) 

    df_trades_results = pd.merge(df_trades_long, df_sl_check, on=['Trade_Date', 'Ticker'], how='left')
    df_trades_results.fillna({'Max_High_in_Window': 0}, inplace=True) 

    # 3.3 Get Exit Close Price at 15:20
    df_exit = df_trading[df_trading.index.time == pd.to_datetime(EXIT_TIME).time()].copy()
    df_exit = df_exit[['Trade_Date', 'Ticker', 'Close']].rename(columns={'Close': 'Exit_Close_Price'})
    df_exit['Ticker'] = df_exit['Ticker'].astype(str)
    df_exit['Trade_Date'] = pd.to_datetime(df_exit['Trade_Date'])

    df_trades_results = pd.merge(df_trades_results, df_exit, on=['Trade_Date', 'Ticker'], how='left')
    df_trades_results.dropna(subset=['Exit_Close_Price'], inplace=True) 

    # 3.4 Final Exit Price Determination
    df_trades_results['SL_Triggered'] = df_trades_results['Max_High_in_Window'] >= df_trades_results['SL_Price']
    df_trades_results['Final_Exit_Price'] = np.where(
        df_trades_results['SL_Triggered'], 
        df_trades_results['SL_Price'],      
        df_trades_results['Exit_Close_Price']
    )

    # 3.5 P&L Calculation
    df_trades_results['Quantity'] = LOT_SIZE
    df_trades_results['Gross_P_L'] = (df_trades_results['Entry_Price'] - df_trades_results['Final_Exit_Price']) * df_trades_results['Quantity']
    df_trades_results['Win_Loss'] = np.where(df_trades_results['Gross_P_L'] > 0, 'Winner', 
                                             np.where(df_trades_results['Gross_P_L'] < 0, 'Loser', 'Breakeven'))

    # Calculate Cumulative P&L (based on daily total P&L)
    df_daily_pnl = df_trades_results.groupby('Trade_Date')['Gross_P_L'].sum().reset_index().rename(columns={'Gross_P_L': 'Daily P&L', 'Trade_Date': 'Entry Date'})
    df_daily_pnl['Cumulative P&L'] = df_daily_pnl['Daily P&L'].cumsum()
    
    # Identify Expiry days for analysis later
    df_daily_pnl['Is_Expiry_Day'] = df_daily_pnl['Entry Date'].apply(lambda x: x in wednesdays)
    
    return df_trades_results, df_daily_pnl


# --- MODULE 3: REPORT GENERATION AND OUTPUT ---

def generate_report(df_trades_results: pd.DataFrame, df_daily_pnl: pd.DataFrame, params: dict):
    """
    Generates the final tradesheet and statistical analysis report.
    """
    # 4.1 Tradesheet (Worksheet 2)
    def extract_strike(ticker): # Re-define locally for clean reporting
        match = re.search(r'(\d+)(CE|PE)', str(ticker))
        return int(match.group(1)) if match else np.nan
        
    tradesheet = pd.DataFrame({
        'Entry Date': df_trades_results['Trade_Date'],
        'Exit Date': df_trades_results['Trade_Date'],
        'Entry Time': df_trades_results['Entry_Time_Actual'],
        'Exit Time': np.where(df_trades_results['SL_Triggered'], 'SL Hit', params['EXIT_TIME']),
        'Option Ticker': df_trades_results['Ticker'],
        'Strike Price': df_trades_results['Ticker'].apply(extract_strike),
        'Option Type': df_trades_results['Option_Type'],
        'Entry Price': df_trades_results['Entry_Price'],
        'Exit Price': df_trades_results['Final_Exit_Price'],
        'Quantity': df_trades_results['Quantity'],
        'Entry Value': df_trades_results['Entry_Price'] * df_trades_results['Quantity'],
        'Exit Value': df_trades_results['Final_Exit_Price'] * df_trades_results['Quantity'],
        'Gross P&L': df_trades_results['Gross_P_L'],
    })
    
    tradesheet = pd.merge(tradesheet, df_daily_pnl[['Entry Date', 'Cumulative P&L']], on='Entry Date', how='left')
    tradesheet['Available capital for each day'] = params['ASSUMED_MARGIN_CAPITAL'] 
    tradesheet['Banknifty underlying close price for that minute'] = 'PLACEHOLDER (Index Data Missing)'


    # 4.2 Statistical Analysis (Worksheet 3)
    stats = {}
    
    # Equity Curve and Drawdown
    df_daily_pnl['Daily_RoC'] = df_daily_pnl['Daily P&L'] / params['ASSUMED_MARGIN_CAPITAL']
    nav_series = (1 + df_daily_pnl['Daily_RoC']).cumprod() * params['INITIAL_NAV']
    df_daily_pnl['NAV'] = nav_series
    df_daily_pnl['High_Water_Mark'] = df_daily_pnl['NAV'].cummax()
    df_daily_pnl['Drawdown'] = (df_daily_pnl['NAV'] - df_daily_pnl['High_Water_Mark']) / df_daily_pnl['High_Water_Mark']

    # CAGR
    total_days = len(df_daily_pnl)
    # Estimate trading days in a year (assuming 252 for Indian markets)
    trading_years = total_days / 252.0 
    ending_nav = df_daily_pnl['NAV'].iloc[-1] 
    stats['CAGR'] = ((ending_nav / params['INITIAL_NAV']) ** (1/trading_years)) - 1
    stats['Max Drawdown'] = df_daily_pnl['Drawdown'].min()

    # Win/Loss Stats (Combined, CE, PE)
    for stat_type in ['', 'CE', 'PE']:
        df_filtered = df_trades_results if stat_type == '' else df_trades_results[df_trades_results['Option_Type'] == stat_type]
        t = len(df_filtered)
        w = len(df_filtered[df_filtered['Win_Loss'] == 'Winner'])
        l = len(df_filtered[df_filtered['Win_Loss'] == 'Loser'])
        label = f' ({stat_type})' if stat_type else ' (Combined)'
        stats[f'Win %{label}'] = (w / t) if t else 0
        stats[f'Number of Winners{label}'] = w
        stats[f'Number of Losers{label}'] = l

    # Average % P&L (Expiry vs. Non-Expiry)
    df_daily_pnl['Day_Type'] = np.where(df_daily_pnl['Is_Expiry_Day'], 'Expiry', 'Non-Expiry')
    df_trades_with_day = pd.merge(df_trades_results[['Trade_Date', 'Option_Type', 'Gross_P_L', 'Entry_Price']], 
                                 df_daily_pnl[['Entry Date', 'Day_Type']], 
                                 left_on='Trade_Date', right_on='Entry Date', how='left')
    # % P&L calculated relative to entry value
    df_trades_with_day['Percent_PL'] = (df_trades_with_day['Gross_P_L'] / (df_trades_with_day['Entry_Price'] * params['LOT_SIZE'])) * 100 

    avg_pl_stats_pivot = df_trades_with_day.groupby(['Day_Type', 'Option_Type'])['Percent_PL'].mean().unstack().fillna(0)
    combined_avg_pl = df_trades_with_day.groupby(['Day_Type'])['Percent_PL'].mean().rename('Average % P&L (Combined)')

    stats_df = pd.DataFrame.from_dict(stats, orient='index', columns=['Value'])
    stats_df['Value'] = stats_df['Value'].apply(lambda x: f'{x*100:.2f}%' if isinstance(x, float) and x > -1 and x < 1 and x != 0 else x)

    # Monthly % P&L Table
    df_monthly_nav = df_daily_pnl.set_index('Entry Date')['NAV'].resample('M').last()
    df_monthly_pnl = df_monthly_nav.pct_change().dropna()
    df_monthly_pnl_table = pd.DataFrame(df_monthly_pnl * 100).rename(columns={'NAV': 'Monthly % P&L'}).reset_index()
    df_monthly_pnl_table['Month'] = df_monthly_pnl_table['Entry Date'].dt.strftime('%Y-%m')

    # --- 5. Export to Excel ---
    excel_filename = 'Harshal_Quant_Backtest_Submission_Modular.xlsx'
    
    with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
        
        # Worksheet 1: Guide/Documentation
        guide_text = f"""
        QUANTITATIVE ANALYSIS REPORT GUIDE AND DOCUMENTATION
        
        Strategy: 09:20 AM Short Strangle on Banknifty
        
        1. Operational Parameters & Assumptions:
            - Entry/Exit Times: {params['ENTRY_TIME']} Entry / {params['EXIT_TIME']} Exit (Hardcoded).
            - Strike Selection Logic: **Highest premium CE and PE options found within the 09:20:00 to 09:25:00 window.**
            - Stop Loss Mechanism: Hard stop-loss of {params['STOP_LOSS_MULTIPLIER']-1:.0%} on entry premium.
            - Capital Assumption (for RoC/NAV): Fixed Margin of ₹{params['ASSUMED_MARGIN_CAPITAL']:,.2f} per day.
        
        2. Performance & Efficiency:
            - Total Trading Days: {total_days}
            - Total Execution Time (Start to End): {time.time() - params['start_time']:.4f} seconds
        """
        guide_df = pd.DataFrame({'Guide/Documentation': [line.strip() for line in guide_text.strip().split('\n')]})
        guide_df.to_excel(writer, sheet_name='1_Guide', index=False)
        
        tradesheet.to_excel(writer, sheet_name='2_Tradesheet', index=False)
        
        # FIX: Ensure the sheet exists before writing formatted text
        stats_df.to_excel(writer, sheet_name='3_Statistics', startrow=0, index=True) 
        
        row_offset = 0
        
        # Write Title and Stats Table
        writer.sheets['3_Statistics'].write(row_offset, 0, 'KEY PERFORMANCE METRICS', writer.book.add_format({'bold': True}))
        # Overwrite the temporary stats_df export with the formatted version
        stats_df.to_excel(writer, sheet_name='3_Statistics', startrow=row_offset + 1, header=True, index=True)
        row_offset += len(stats_df) + 3

        # Write Average P&L Breakdown
        writer.sheets['3_Statistics'].write(row_offset, 0, 'AVERAGE % P&L BREAKDOWN (EXPIRY VS. NON-EXPIRY)', writer.book.add_format({'bold': True}))
        avg_pl_stats_pivot.to_excel(writer, sheet_name='3_Statistics', startrow=row_offset + 1, header=True, index=True)
        row_offset += len(avg_pl_stats_pivot) + 4
        
        combined_avg_pl.to_excel(writer, sheet_name='3_Statistics', startrow=row_offset + 1, header=True)
        row_offset += len(combined_avg_pl) + 3

        # Write Monthly P&L
        writer.sheets['3_Statistics'].write(row_offset, 0, 'MONTHLY % P&L TABLE (FROM NAV)', writer.book.add_format({'bold': True}))
        df_monthly_pnl_table[['Month', 'Monthly % P&L']].to_excel(writer, sheet_name='3_Statistics', startrow=row_offset + 1, header=True, index=False)
        row_offset += len(df_monthly_pnl_table) + 4

        # Write NAV/Drawdown for Plotting
        writer.sheets['3_Statistics'].write(row_offset, 0, 'NAV & DRAWDOWN DATA (FOR PLOTTING)', writer.book.add_format({'bold': True}))
        df_daily_pnl[['Entry Date', 'NAV', 'Drawdown']].to_excel(writer, sheet_name='3_Statistics', startrow=row_offset + 1, header=True, index=False)
        
    print(f"\n--- Strategy Report Generated ---")
    print(f"Output saved to: {excel_filename}")


# --- MAIN EXECUTION BLOCK ---

if __name__ == "__main__":
    start_time = time.time()
    
    # 0.1 Strategy Parameters defined in a single dict
    strategy_params = {
        'ENTRY_TIME': '09:20:00',
        'EXIT_TIME': '15:20:00',
        'PREMIUM_TARGET': 30.0,
        'STOP_LOSS_MULTIPLIER': 1.50,
        'LOT_SIZE': 15,
        'INITIAL_NAV': 100.0,
        'ASSUMED_MARGIN_CAPITAL': 200000.0,
        'start_time': start_time # Pass start time for reporting
    }

    try:
        # Step 1: Prepare Data
        print("1. Preparing data...")
        df_trading_data = prepare_data('Options_data_2023.csv')
        
        # Step 2: Execute Strategy and Calculate P&L
        print("2. Executing strategy and calculating P&L...")
        df_trades, df_pnl = execute_strangle_strategy(df_trading_data, strategy_params)
        
        # Step 3: Generate and Save Report
        print("3. Generating final report...")
        generate_report(df_trades, df_pnl, strategy_params)

    except Exception as e:
        print(f"\nFATAL BACKTEST EXCEPTION: {e}")
        print("Backtest failed due to a critical error.")

1. Preparing data...
2. Executing strategy and calculating P&L...
3. Generating final report...

--- Strategy Report Generated ---
Output saved to: Harshal_Quant_Backtest_Submission_Modular.xlsx
